# 1. Installing dependencies

In [2]:
ZIP_NAME = "BlueProp.v2i.yolov5pytorch.zip"


In [3]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 16199, done.
remote: Counting objects: 100% (107/107), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 16199 (delta 31), reused 74 (delta 13), pack-reused 16092
Receiving objects: 100% (16199/16199), 15.05 MiB | 25.44 MiB/s, done.
Resolving deltas: 100% (11053/11053), done.


In [4]:
!pip install -r yolov5/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.2/663.2 kB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.4 MB/s eta 0:00:00
  Attempting uninstall: Pillow
    Found existing installation: Pillow 9.4.0
    Uninstalling Pillow-9.4.0:
      Successfully uninstalled Pillow-9.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imageio 2.31.6 requires pillow<10.1.0,>=8.3.2, but you have pillow 10.1.0 which is incompatible.


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
def findLocation(filename: str, exclude: list = []):
    """Find the location of a file in the user's home directory\n\nparam: filename - The name of the file to search for.\nparam: exclude - A list of strings to exclude from the search.\n\nreturn: The path to the file if it is found, None otherwise."""
    import os

    def hasExceptions(path: str) -> bool:
        path = path.split(os.path.sep)
        return any(i in path for i in exclude)

    for root, dirs, files in os.walk(os.path.expanduser('/')):
        if filename in files and hasExceptions(root) is False:
            return os.path.join(root, filename)

    return None

def loadData(name: str):
  location: str = findLocation(name)
  if location is not None:
    !unzip -o {location} -d ./data
  else: raise Exception("ZIP file couldn't be found")

In [7]:
def fixYaml(yaml_path: str) -> None:
    import os
    """Fixes the YAML file to make it compatible with the training process by replacing the relative path with an absolute one.\n\nparam: yaml_path - The path to the YAML file."""
    with open(yaml_path, 'r', encoding='utf-8') as yaml_file:
        yaml_data = yaml_file.read()

    yaml_data = yaml_data.split('\n')
    os.chdir(os.path.dirname(yaml_path))
    for i in range(3):
      path_start = yaml_data[i].find(" ") + 1
      yaml_data[i] = yaml_data[i][:path_start] + os.path.abspath(yaml_data[i][path_start:])

    yaml_data = "\n".join(yaml_data)
    with open(yaml_path, 'w') as yaml_file:
        yaml_file.write(yaml_data)

def getLastModel():
  import os
  from google.colab import files

  if not os.path.isdir("/content/yolov5/runs/train"):
    raise Exception("No models found")

  items = os.listdir('/content/yolov5/runs/train')
  folders = [item for item in items if os.path.isdir(os.path.join('/content/yolov5/runs/train', item))]

  model_path = os.path.join('/content/yolov5/runs/train/exp' + ('' if len(folders) == 1 else str(len(folders))), 'weights', 'best.pt')
  if os.path.exists(model_path):
    files.download(model_path)
  else: raise Exception("No model found in the last experiment")

In [8]:
loadData(ZIP_NAME)

Archive:  /content/drive/MyDrive/BlueProp.v2i.yolov5pytorch.zip
 extracting: ./data/README.dataset.txt  
 extracting: ./data/README.roboflow.txt  
 extracting: ./data/data.yaml        
   creating: ./data/test/
   creating: ./data/test/images/
 extracting: ./data/test/images/WIN_20231227_22_19_00_Pro_jpg.rf.a6be41a7b1665aca52eea0ddc2dd8dd6.jpg  
 extracting: ./data/test/images/WIN_20231227_22_19_07_Pro_jpg.rf.eeca0f90c0b02f8d2b6482ab3d6d457f.jpg  
 extracting: ./data/test/images/WIN_20231227_22_23_29_Pro_jpg.rf.bd0c90ccbad3a4e0b4887638babb6926.jpg  
 extracting: ./data/test/images/WIN_20231227_22_28_26_Pro_jpg.rf.b29c16004e5f79fd65b79c85165c33fd.jpg  
 extracting: ./data/test/images/WIN_20231227_22_31_28_Pro_jpg.rf.4ff49b0948c46e45254e9e062b2e983e.jpg  
 extracting: ./data/test/images/WIN_20231227_22_31_33_Pro_jpg.rf.96557a1af46da531939362a4abbe590d.jpg  
 extracting: ./data/test/images/WIN_20231227_22_39_00_Pro_jpg.rf.fa3e75c8fe4a87883df5687e5c14de85.jpg  
 extracting: ./data/test/ima

# 3. Train from scratch

In [10]:
yaml_location = findLocation("data.yaml", "drive")
#Make sure to edit data.yaml to the right paths (try to fix soon)
fixYaml(yaml_location)
!cd {yaml_location[:-10]} && python /content/yolov5/train.py --img 640 --batch -1 --epochs 300 --data {yaml_location} --weights yolov5s.pt

2024-01-02 03:26:38.510100: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-02 03:26:38.510155: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-02 03:26:38.511601: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=, data=/content/data/data.yaml, hyp=../yolov5/data/hyps/hyp.scratch-low.yaml, epochs=300, batch_size=-1, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False,

In [11]:
getLastModel()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
from google.colab import files
import shutil

shutil.make_archive("/content/yolo_experiment", "zip", "/content/yolov5/runs/train/exp4")
files.download("/content/yolo_experiment.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>